# Query pool liquidity on a decentralised exchange

In this notebook, we show how to get historical (past) available liquidity on [PancakeSwap](https://tradingstrategy.ai/trading-view/binance/pancakeswap-v2) pool.
In this example, we use [BNB-BUSD pair](https://tradingstrategy.ai/trading-view/binance/pancakeswap-v2/bnb-busd). We also show how to get the liquidity added and removed during the time period.

## About liquidity

[Read the introduction blog post for liquidity analysis](https://tradingstrategy.ai/blog/announcing-support-for-liquidity-charts)
so you understand how the liquidity is presented.

## Getting started

[Please read and complete the Getting started tutorial first](https://tradingstrategy.ai/docs/programming/examples/getting-started.html).

First let's import libraries and initialise our Trading Strategy dataset client.

In [2]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

ModuleNotFoundError: No module named 'tradingstrategy'

## Fetching datasets

Here we fetch the liquidity data for

* [BNB-BUSD trading pair](https://tradingstrategy.ai/trading-view/binance/pancakeswap-v2/bnb-busd)
* [Trading on PancakeSwap](https://tradingstrategy.ai/trading-view/binance/pancakeswap-v2)
* [On Binance Smart Chain](https://tradingstrategy.ai/trading-view/binance/)

To make this notebook run fast, we are operating on the historical weekly candles (7 days) to keep the
download size of datasets smaller.

In [32]:

from tradingstrategy.chain import ChainId
from tradingstrategy.liquidity import GroupedLiquidityUniverse
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.timebucket import TimeBucket

#: We use weekly liquidity candles for the calculations
CANDLE_TIMEFRAME = TimeBucket.d7

# Download exchange database
exchange_universe = client.fetch_exchange_universe()

# Download trading pair database
raw_pairs = client.fetch_pair_universe().to_pandas()

# Download liquidity OHLC candles
raw_liquidity_samples = client.fetch_all_liquidity_samples(CANDLE_TIMEFRAME).to_pandas()

# Create a Python helper to manipulate trading pairs
pair_universe = PandasPairUniverse(raw_pairs)

# Create a Python helper to manipulate pair liquidity data
liquidity_universe = GroupedLiquidityUniverse(raw_liquidity_samples)

# Do some test calculations for a single pair
# Note that PancakeSwap has two different deployments:
# you most likely want v2
our_exchange = exchange_universe.get_by_chain_and_slug(ChainId.bsc, "pancakeswap-v2")
assert our_exchange, "Could not find the DEX"

# Query the pair metadata.
# Note that PancakeSwap operates on Wrapped BNB (WBNB), not native BNB token
base_token = "WBNB"
quote_token = "BUSD"
our_pair = pair_universe.get_one_pair_from_pandas_universe(our_exchange.exchange_id, "WBNB", "BUSD", pick_by_highest_vol=True)

assert our_pair, "Pair not found"

print(f"Trading pair {base_token}-{quote_token} id is {our_pair.pair_id}, the pool contract address is {our_pair.address}")

Trading pair WBNB-BUSD id is 1015916, the pool contract address is 0x58f876857a02d6762e0101bb5c46a8c1ed44dc16


## Querying historical liquidity data

Here we get the historical available liquidity at 2022-01-01.

Unlike many other examples, or the underlying DEX protocol, here we operate on US Dollar amounts.
[Any token price is converted to US Dollar through Trading Strategy reference pricing mechanism](https://tradingstrategy.ai/docs/programming/referenceprice.html).

[See also XYLiquidity data structure documentation](https://tradingstrategy.ai/docs/programming/api/liquidity.html)
for available values.

In [1]:
import pandas as pd

# December 2021
when = pd.Timestamp("2021-12-01")

liquidity_samples = liquidity_universe.get_liquidity_samples_by_pair(our_pair.pair_id)
if liquidity_samples is None:
    raise AssertionError(f"The universe does not contain liquidity data for pair {our_pair}")

# Get all liquidity OHLC past the certain timestamp.
# Thus, we query the first available liquidity sample *after* the timestamp.
ranged_samples = liquidity_samples[when:]

if len(ranged_samples) == 0:
    raise AssertionError(f"Pair {our_pair} had no liquidity before {when}")

# Get the first sample in the series
# This presents the liquidity at the first available sampled moment
# since the timestamp.
first_sample = ranged_samples.iloc[0]

# Convert liquidity from USD to token
token_liquidity = first_sample.open * first_sample.exchange_rate

print(f"""
Liquidity sampled at: {first_sample.timestamp}
Liquidity sample period: {CANDLE_TIMEFRAME.value}
Liquidity at the begin: {first_sample.open:,.2f} USD
Liquidity at the end: {first_sample.close:,.2f} USD
Liquidity at the begin, expressed as the quote token: {token_liquidity:.4f} {our_pair.quote_token_symbol}
Liquidity added during the period: {first_sample.add_volume:,.2f} USD
Liquidity removed during the period: {first_sample.remove_volume:,.2f} USD
""")

NameError: name 'liquidity_universe' is not defined

## Further info and sources of potential errors

See also [price impact example](https://tradingstrategy.ai/docs/programming/examples/price-impact.html).